In [ ]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re
import math

In [ ]:
data = pd.read_csv("hn2018_2019.csv")

In [ ]:
def dater(datetim,integer):
    date,time = datetim.split(" ")
    year,month,day = date.split("-")
    hour,minute,second = time.split(":")
    second = int(second)+ int(minute)*60 + int(hour)*60*60
    if integer==1:
        return year
    elif integer==2:
        return month
    elif integer==3:
        return day
    else:
        return second

In [ ]:
data["year"] = data['Created At'].map(lambda x: dater(x,1))
data["month"] = data['Created At'].map(lambda x: dater(x,2))
data["day"] = data['Created At'].map(lambda x: dater(x,3))
data["second"] = data['Created At'].map(lambda x: dater(x,4))

In [ ]:
df_2018 = data[data["year"]=='2018'][['Title','Post Type']].copy()

In [ ]:
df_2018['Title'] = df_2018['Title'].map(lambda x:x.lower())

## Adding a column of tokenized words in the dataframe

In [ ]:
df_2018['tokenized_title'] = df_2018['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())

In [ ]:
data.head()

In [ ]:
df_2018.head()

## Calculating frequency of each word and given their conditional post type

In [ ]:
def input_phrase(phrase,typ,dict_words):
    for i in phrase:
        if i not in dict_words:
            dict_words[i] = {'story':0,'ask_hn':0,'show_hn':0,'poll':0}
        dict_words[i][typ] += 1
    return dict_words

In [ ]:
d = {}
totalNumberOfWords = 0
uniqueWords = 0
for i in range(len(df_2018['tokenized_title'])):
    d = input_phrase(df_2018['tokenized_title'][i],df_2018['Post Type'][i],d)
    totalNumberOfWords += len(df_2018['tokenized_title'][i])

In [ ]:
words_2018 = pd.DataFrame(d)
words_2018 = words_2018.transpose()
delta = 0.5
uniqueWords = len(words_2018)
absentWordConditionalProbability = {}
for className in ['ask_hn', 'poll', 'show_hn', 'story']:
    wordsPerClass = np.sum(words_2018[className])
    conditionalClassLabel = 'prob_' + className
    words_2018[conditionalClassLabel] = words_2018[className].map(lambda x: (int(x) + delta)/( wordsPerClass + delta*uniqueWords ))    
    absentWordConditionalProbability[conditionalClassLabel] = delta/( wordsPerClass + delta*uniqueWords )
    
writeWords2018 = words_2018.copy()
words_2018 = words_2018.transpose()

In [ ]:
writeWords2018.index.name = 'TokenName'
writeWords2018 = writeWords2018.reset_index()
writeWords2018 = writeWords2018.sort_values(by ='TokenName')
writeWords2018 = writeWords2018.reset_index()
cols = ['TokenName', 'story', 'prob_story', 'ask_hn', 'prob_ask_hn', 'show_hn', 'prob_show_hn', 'poll', 'prob_poll']
writeWords2018 = writeWords2018[cols]
writeWords2018.index += 1
filename = 'model-2018.txt'
writeWords2018.to_csv(filename, header = None, index = True, sep = ' ', mode = 'w')
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line.replace(' ', '  ') for line in lines]
with open(filename, 'w') as f:
    f.writelines(lines)

In [ ]:
words_2018.head(10)

## PART 2

In [ ]:
priorProbabilities = {'prob_story':0,'prob_ask_hn':0,'prob_show_hn':0,'prob_poll':0}
unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
for index in range(len(unique)):
    priorProbabilities['prob_' + unique[index]] = counts[index]/np.sum(counts)
print(priorProbabilities)

In [ ]:
conditionalProbabilities = words_2018.drop(['story', 'poll', 'show_hn', 'ask_hn'])
conditionalProbabilities.head(10)

In [ ]:
def getConditionalProbability(word,className):
    try:
        return conditionalProbabilities[word][className]
    except:
        return absentWordConditionalProbability[className]

In [ ]:
def generateScore(arrayOfTokenizedTitle,className):
    score = np.zeros(len(arrayOfTokenizedTitle)) + math.log10(priorProbabilities[className])
    for index in range(len(arrayOfTokenizedTitle)):
        wordsList = arrayOfTokenizedTitle[index]
        for word in wordsList:
            score[index] += math.log10(getConditionalProbability(word,className))
    return score

In [ ]:
testData = data[data["year"]=='2019'][['Title','Post Type']].copy()
testData['Title'] = testData['Title'].map(lambda x:x.lower())
testData['tokenized_title'] = testData['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())
testData = testData.reset_index()

In [ ]:
testData.head()

In [ ]:
for className in priorProbabilities.keys():
    testData[className] = generateScore(testData['tokenized_title'],className)
testData.head()

In [ ]:
columnNamesExchange = {}
for className in ['ask_hn', 'poll', 'show_hn', 'story']:
    columnNamesExchange['prob_' + className] = className
dfObj = testData[['prob_ask_hn', 'prob_poll', 'prob_show_hn', 'prob_story']].copy()
dfObj = dfObj.rename(columns=columnNamesExchange)
dfObj['predicted'] = dfObj.idxmax(axis=1)
dfObj = pd.concat([dfObj, testData[['Title', 'Post Type']]], axis=1)
dfObj['comparision'] = dfObj['predicted'] == dfObj['Post Type']
cols = ['Title', 'story', 'ask_hn', 'show_hn', 'poll', 'Post Type', 'comparision']
dfObj = dfObj[cols]
dfObj.head()

In [ ]:
filename = 'result.txt'
dfObj.to_csv(filename, header = None, index = False, sep = ' ', mode = 'w')
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line.replace(' ', '  ') for line in lines]
with open(filename, 'w') as f:
    f.writelines(lines)

In [ ]:
dfObj.comparision.value_counts()